<a href="https://colab.research.google.com/github/esuda/mineiracao_dados_complexos/blob/master/Big%20Data/Big_Data_010_Aula_02_Hadoop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dados de Entrada

*   Abra o link dos dados: https://tinyurl.com/bd10-gut
*   Clique em "Adicionar ao Drive"


In [ ]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


#Instalação e Configuração do Hadoop Single Node

## Instalação de pacotes

In [ ]:
!wget -q https://downloads.apache.org/hadoop/common/hadoop-3.3.0/hadoop-3.3.0.tar.gz
!tar xf hadoop-3.3.0.tar.gz
!apt-get -qq -y install openssh-server > /dev/null

In [ ]:
!service ssh restart

 * Restarting OpenBSD Secure Shell server sshd
   ...done.


In [ ]:
!ssh-keygen -t rsa -P '' -f ~/.ssh/id_rsa
!cat ~/.ssh/id_rsa.pub >> ~/.ssh/authorized_keys
!chmod 0600 ~/.ssh/authorized_keys

Generating public/private rsa key pair.
Created directory '/root/.ssh'.
Your identification has been saved in /root/.ssh/id_rsa.
Your public key has been saved in /root/.ssh/id_rsa.pub.
The key fingerprint is:
SHA256:el+qSFaNPpQrvKaLpx4iDbzt9L9U9/BLI2OTvSPXPPU root@ac1391d39c3a
The key's randomart image is:
+---[RSA 2048]----+
|                 |
|                 |
|                 |
|.        +       |
|..      S..o     |
| oo  . =... *   .|
|o.oo  *.=  *.*o o|
|..ooooo= o.+=o++E|
| .++o=+oo.o oo. .|
+----[SHA256]-----+


  ## Arquivos de Configuração do Hadoop

In [ ]:
%%file /content/hadoop-3.3.0/etc/hadoop/core-site.xml

<configuration>
    <property>
        <name>fs.defaultFS</name>
        <value>hdfs://localhost:9000</value>
    </property>
</configuration>

Overwriting /content/hadoop-3.3.0/etc/hadoop/core-site.xml


In [ ]:
%%file /content/hadoop-3.3.0/etc/hadoop/hdfs-site.xml

<configuration>
    <property>
        <name>dfs.replication</name>
        <value>1</value>
    </property>
</configuration>

Overwriting /content/hadoop-3.3.0/etc/hadoop/hdfs-site.xml


In [ ]:
%%file hadoop-3.3.0/etc/hadoop/hadoop-env.sh

export HDFS_NAMENODE_USER="root"
export HDFS_DATANODE_USER="root"
export HDFS_SECONDARYNAMENODE_USER="root"
export YARN_RESOURCEMANAGER_USER="root"
export YARN_NODEMANAGER_USER="root"
export JAVA_HOME=/usr/lib/jvm/java-11-openjdk-amd64/
export PATH=$PATH:/content/hadoop-3.3.0/bin:/content/hadoop-3.3.0/sbin


Overwriting hadoop-3.3.0/etc/hadoop/hadoop-env.sh


In [ ]:
import os
os.environ["PATH"] += os.pathsep + "/content/hadoop-3.3.0/bin"

## Preparação do Sistema de Arquivos HDFS

In [ ]:
!hdfs namenode -format

2021-10-30 11:48:11,309 INFO namenode.NameNode: STARTUP_MSG: 
/************************************************************
STARTUP_MSG: Starting NameNode
STARTUP_MSG:   host = ac1391d39c3a/172.28.0.2
STARTUP_MSG:   args = [-format]
STARTUP_MSG:   version = 3.3.0
STARTUP_MSG:   classpath = /content/hadoop-3.3.0/etc/hadoop:/content/hadoop-3.3.0/share/hadoop/common/lib/log4j-1.2.17.jar:/content/hadoop-3.3.0/share/hadoop/common/lib/commons-text-1.4.jar:/content/hadoop-3.3.0/share/hadoop/common/lib/re2j-1.1.jar:/content/hadoop-3.3.0/share/hadoop/common/lib/accessors-smart-1.2.jar:/content/hadoop-3.3.0/share/hadoop/common/lib/jetty-security-9.4.20.v20190813.jar:/content/hadoop-3.3.0/share/hadoop/common/lib/jcip-annotations-1.0-1.jar:/content/hadoop-3.3.0/share/hadoop/common/lib/jetty-webapp-9.4.20.v20190813.jar:/content/hadoop-3.3.0/share/hadoop/common/lib/jaxb-impl-2.2.3-1.jar:/content/hadoop-3.3.0/share/hadoop/common/lib/jsr311-api-1.1.1.jar:/content/hadoop-3.3.0/share/hadoop/common/lib/k

In [ ]:
!hadoop-3.3.0/sbin/start-dfs.sh

Starting namenodes on [localhost]
localhost: Warning: Permanently added 'localhost' (ECDSA) to the list of known hosts.
Starting datanodes
Starting secondary namenodes [ac1391d39c3a]
ac1391d39c3a: Warning: Permanently added 'ac1391d39c3a,172.28.0.2' (ECDSA) to the list of known hosts.


# HDFS e Hadoop Streaming

## Cópia e Acesso a arquivos no HDFS

In [ ]:
!hdfs help

Usage: hdfs [OPTIONS] SUBCOMMAND [SUBCOMMAND OPTIONS]

  OPTIONS is none or any of:

--buildpaths                       attempt to add class files from build tree
--config dir                       Hadoop config directory
--daemon (start|status|stop)       operate on a daemon
--debug                            turn on shell script debug mode
--help                             usage information
--hostnames list[,of,host,names]   hosts to use in worker mode
--hosts filename                   list of hosts to use in worker mode
--loglevel level                   set the log4j level for this command
--workers                          turn on worker mode

  SUBCOMMAND is one of:


    Admin Commands:

cacheadmin           configure the HDFS cache
crypto               configure HDFS encryption zones
debug                run a Debug Admin to execute HDFS debug commands
dfsadmin             run a DFS admin client
dfsrouteradmin       manage Router-based federation
ec                   run a HD

In [ ]:
!hdfs dfs  -usage put

Usage: hadoop fs [generic options] -put [-f] [-p] [-l] [-d] <localsrc> ... <dst>


In [ ]:
!hdfs dfs -mkdir /mdc

In [ ]:
!hdfs dfs -ls /

Found 1 items
drwxr-xr-x   - root supergroup          0 2021-10-30 11:51 /mdc


In [ ]:
!ls /

bin	 datalab  home	 lib64	opt	    root  srv		     tmp    var
boot	 dev	  lib	 media	proc	    run   sys		     tools
content  etc	  lib32  mnt	python-apt  sbin  tensorflow-1.15.2  usr


In [ ]:
!hdfs dfs -du -h /

0  0  /mdc


In [ ]:
!hdfs dfs -df -h 

Filesystem                Size  Used  Available  Use%
hdfs://localhost:9000  107.7 G  24 K     59.9 G    0%


In [ ]:
!hdfs dfs -mkdir /bigdata


In [ ]:
!hdfs dfs -mkdir -p /um/dois/tres

In [ ]:
!hdfs dfs -ls -R /

drwxr-xr-x   - root supergroup          0 2021-10-30 11:52 /bigdata
drwxr-xr-x   - root supergroup          0 2021-10-30 11:51 /mdc
drwxr-xr-x   - root supergroup          0 2021-10-30 11:52 /um
drwxr-xr-x   - root supergroup          0 2021-10-30 11:52 /um/dois
drwxr-xr-x   - root supergroup          0 2021-10-30 11:52 /um/dois/tres


In [ ]:
!hdfs dfs -rm -r /um

Deleted /um


In [ ]:
!hdfs dfs -ls -R /

drwxr-xr-x   - root supergroup          0 2021-10-30 11:52 /bigdata
drwxr-xr-x   - root supergroup          0 2021-10-30 11:51 /mdc


In [ ]:
!hdfs dfs -put /content/drive/My%20Drive/gut-pt/small /mdc

In [ ]:
!hdfs dfs -put /content/drive/My%20Drive/gut-pt/all /mdc

In [ ]:
!hdfs dfs -ls /mdc/small/

Found 5 items
-rw-r--r--   1 root supergroup     132230 2021-10-30 11:53 /mdc/small/u-27350-8
-rw-r--r--   1 root supergroup      80852 2021-10-30 11:53 /mdc/small/u-29040-8
-rw-r--r--   1 root supergroup     331033 2021-10-30 11:53 /mdc/small/u-33056-8
-rw-r--r--   1 root supergroup     403704 2021-10-30 11:53 /mdc/small/u-54829-8
-rw-r--r--   1 root supergroup     493358 2021-10-30 11:53 /mdc/small/u-55682-8


In [ ]:
!hdfs dfs -cp /mdc/small/ /mdc_copia

In [ ]:
!hdfs dfs -ls /mdc_copia

In [ ]:
!hdfs dfs -get /mdc_copia/ .

In [ ]:
!ls ./mdc_copia

u-27350-8  u-29040-8  u-33056-8  u-54829-8  u-55682-8


In [ ]:
!hdfs dfs -cat /mdc/small/u-54829-8 | head

Project Gutenberg's Memorias Postumas de Braz Cubas, by Machado de Assis

This eBook is for the use of anyone anywhere in the United States and most
other parts of the world at no cost and with almost no restrictions
whatsoever.  You may copy it, give it away or re-use it under the terms of
the Project Gutenberg License included with this eBook or online at
www.gutenberg.org.  If you are not located in the United States, you'll have
to check the laws of the country where you are located before using this ebook.

Title: Memorias Postumas de Braz Cubas
cat: Unable to write to output stream.


In [ ]:
!hdfs dfs -cat /mdc/small/u-54829-8 | tail

edition.

Most people start at our Web site which has the main PG search
facility: www.gutenberg.org

This Web site includes information about Project Gutenberg-tm,
including how to make donations to the Project Gutenberg Literary
Archive Foundation, how to help produce our new eBooks, and how to
subscribe to our email newsletter to hear about new eBooks.



## Scripts Map e Reduce

In [ ]:
%%file count_map.py

import sys
import re

for line in sys.stdin:
  line = re.sub('[^a-z ]', ' ', line.lower())
  for word in line.split() :
    print(word+"\t"+str(1))


Writing count_map.py


In [ ]:
%%file count_reduce.py

import sys

current_word = None
count_count = 0

for line in sys.stdin:
  word, value = line.split('\t')
  count = int(value)
  if current_word != word :
    # se a palavra de entrada é diferente da palavra sendo contada
    if current_word :
      # se a palavra sendo contada não é nula
      print(current_word+"\t"+str(count_count))
    # muda a palavra sendo contada
    current_word = word
    count_count = count
  else :
    # se a palavra de entrada é a mesma que está sendo contada
    count_count = count_count + count

# imprime a última palavra
if current_word :
  print(current_word+"\t"+str(count_count))


Writing count_reduce.py


In [ ]:
!cat "/content/drive/My Drive/gut/small/"* | python count_map.py | sort | python count_reduce.py


## Execução com Hadoop Streaming

In [ ]:
!hdfs dfs -rm -r /mdc/output/

In [ ]:
!hadoop jar /content/hadoop-3.3.0/share/hadoop/tools/lib/hadoop-streaming-3.3.0.jar \
  -files 'count_map.py,count_reduce.py' \
  -mapper 'python count_map.py' \
  -reducer 'python count_reduce.py' \
  -input /mdc/small \
  -output /mdc/output


2021-10-30 12:02:13,626 INFO impl.MetricsConfig: Loaded properties from hadoop-metrics2.properties
2021-10-30 12:02:13,769 INFO impl.MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
2021-10-30 12:02:13,769 INFO impl.MetricsSystemImpl: JobTracker metrics system started
2021-10-30 12:02:13,794 WARN impl.MetricsSystemImpl: JobTracker metrics system already initialized!
2021-10-30 12:02:14,242 INFO mapred.FileInputFormat: Total input files to process : 5
2021-10-30 12:02:14,261 INFO mapreduce.JobSubmitter: number of splits:5
2021-10-30 12:02:14,598 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local247783474_0001
2021-10-30 12:02:14,598 INFO mapreduce.JobSubmitter: Executing with tokens: []
2021-10-30 12:02:14,970 INFO mapred.LocalDistributedCacheManager: Localized file:/content/count_map.py as file:/tmp/hadoop-root/mapred/local/job_local247783474_0001_0681a31a-fc62-447e-b29d-f921cb4510ce/count_map.py
2021-10-30 12:02:15,025 INFO mapred.LocalDistributedCac

In [ ]:
!hadoop jar /content/hadoop-3.3.0/share/hadoop/tools/lib/hadoop-streaming-3.3.0.jar \
  -files 'count_map.py,count_reduce.py' \
  -mapper 'python count_map.py' \
  -reducer 'python count_reduce.py' \
  -input /mdc/all \
  -output /mdc/output-all


Streaming output truncated to the last 5000 lines.
	File System Counters
		FILE: Number of bytes read=2325837
		FILE: Number of bytes written=137257825
		FILE: Number of read operations=0
		FILE: Number of large read operations=0
		FILE: Number of write operations=0
		HDFS: Number of bytes read=87208191
		HDFS: Number of bytes written=0
		HDFS: Number of read operations=971
		HDFS: Number of large read operations=0
		HDFS: Number of write operations=1
		HDFS: Number of bytes read erasure-coded=0
	Map-Reduce Framework
		Map input records=716
		Map output records=5861
		Map output bytes=44434
		Map output materialized bytes=56162
		Input split bytes=91
		Combine input records=0
		Spilled Records=5861
		Failed Shuffles=0
		Merged Map outputs=0
		GC time elapsed (ms)=10
		Total committed heap usage (bytes)=707788800
	File Input Format Counters 
		Bytes Read=35903
2021-10-30 12:10:05,679 INFO mapred.LocalJobRunner: Finishing task: attempt_local100453192_0001_m_000483_0
2021-10-30 12:10:05,6

In [ ]:
!hdfs dfs -ls /mdc/all/

Found 539 items
-rw-r--r--   1 root supergroup      99749 2021-10-30 12:04 /mdc/all/u-11299-8
-rw-r--r--   1 root supergroup     114107 2021-10-30 12:04 /mdc/all/u-12579
-rw-r--r--   1 root supergroup     115745 2021-10-30 12:04 /mdc/all/u-12579-8
-rw-r--r--   1 root supergroup     128832 2021-10-30 12:04 /mdc/all/u-13092
-rw-r--r--   1 root supergroup     130725 2021-10-30 12:04 /mdc/all/u-13092-8
-rw-r--r--   1 root supergroup     123105 2021-10-30 12:04 /mdc/all/u-13093
-rw-r--r--   1 root supergroup     124942 2021-10-30 12:04 /mdc/all/u-13093-8
-rw-r--r--   1 root supergroup     125691 2021-10-30 12:04 /mdc/all/u-13630
-rw-r--r--   1 root supergroup     127460 2021-10-30 12:04 /mdc/all/u-13630-8
-rw-r--r--   1 root supergroup     160251 2021-10-30 12:04 /mdc/all/u-14296
-rw-r--r--   1 root supergroup     122409 2021-10-30 12:04 /mdc/all/u-14296-8
-rw-r--r--   1 root supergroup     326018 2021-10-30 12:04 /mdc/all/u-14503-8
-rw-r--r--   1 root supergroup     123266 2021-10-30 12:04

In [ ]:
!hdfs dfs -cat /mdc/output/part* | tail -n 20